# Create chm_fp collection in genra_db

In [ ]:
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *

DB = openMongo(db='genra_dev_v4')


In [ ]:
DB.collection_names()
DB.compounds.find_one().keys()

In [ ]:
DB.compounds.find_one(dict(name='Bisphenol A'))['dsstox_sid']

In [ ]:
from chm.mkfp import *

In [ ]:
makeFP('DTXSID7020182', col_comp=DB.compounds,col_chm_fp=DB.chm_fp,save=True)

In [ ]:
for C in DB.compounds.find():
    makeFP()

In [ ]:
for i in ['dsstox_sid']: DB.chm_fp.create_index(i)

In [ ]:
DB.chm_fp.count()

# Serial FP generation


In [ ]:
import datetime
from chm.mkfp import *

N = DB.compounds.count() - DB.chm_fp.count()
print "Start ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",N
    
k=0
for C in DB.compounds.find(no_cursor_timeout=True):
    k+=1
    makeFP(C['dsstox_sid'], col_comp=DB.compounds,col_chm_fp=DB.chm_fp,
           save=True,replace=False)
        
    if k%1000==0:
        print k,'/',N,datetime.datetime.now().strftime("%H:%M:%S")
        
print "Ended ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",N

# Parallel FP generation

In [ ]:
import ipyparallel as PP

RC=PP.Client(profile='chiron_pm')
d_view=RC[:]
lb_view = RC.load_balanced_view()
lb_view.block = True
x=file("../env.py",'r').read()
d_view.execute(x)

In [ ]:
DB.chm_fp.count()

In [ ]:
d_view.execute("from chm.mkfp import *")


In [ ]:
DB.chm_fp.find_one().keys()

In [ ]:
#DB.chm_fp.delete_many({})
for i in ['name','mrgn.ds','mrgn.n','httr.n','httr.ds','casrn',
          'dsstox_cid','dsstox_sid']:
    print i
    DB.chm_fp.create_index(i)


In [ ]:
from threading import Thread
SID = []

def procCurs(curs,res):
    for C in curs:
        makeFP(C['dsstox_sid'], col_comp=DB.compounds,col_chm_fp=DB.chm_fp,
               save=True,replace=False)
        
        
Curs = DB.compounds.parallel_scan(50)
TH0  = [Thread(target=procCurs,args=(curs,SID)) for curs in Curs]
for th in TH0: th.start()
for th in TH0: th.join()


## How many chemicals left to do

In [ ]:
DONE = DB.chm_fp.distinct('dsstox_sid')
WORK = []

for C in DB.compounds.find():
    if DB.chm_fp.find(dict(dsstox_sid=C['dsstox_sid'])).count()>0: 
        continue
    else:
        WORK.append(C['dsstox_sid'])
#list(set(DB.compounds.distinct('dsstox_sid')).difference(DONE))
len(WORK)

## Run in parallel

In [ ]:
import datetime
from chm.mkfp import *

            
print "Start ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK)

lb_view.map(lambda dtxsid: makeFP(dtxsid, col_comp=DB.compounds,col_chm_fp=DB.chm_fp,save=True),WORK)


print "End ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK)



In [ ]:
DB.chm_fp.find_one()